In [175]:
import json
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import sigmoid_kernel
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import time
import string
import nltk

In [176]:
#papers_df is pandas dataframe object
papers_df = pd.read_json('temp4.json', lines=True)

papers_df.head()

#papers_df = pd.read_json('NIP_DataSet/papers_2K.json',lines=True)
papers_df.rename(columns = {"paper_text" : "summary"},inplace=True)
papers_df.head()

,day,id,month,summary,title,year,Topic1,Topic2,Topic3,Author1,Author2,textLink,pdfLink
0,1,1802.00209v1,2,We propose an architecture for VQA which utili...,Dual Recurrent Attention Units for Visual Ques...,2018,cs.AI,cs.CL,cs.CV,Ahmed Osman,Wojciech Samek,http://arxiv.org/abs/1802.00209v1,http://arxiv.org/pdf/1802.00209v1
1,12,1603.03827v1,3,Recent approaches based on artificial neural n...,Sequential Short-Text Classification with Recu...,2016,cs.CL,cs.AI,cs.LG,Ji Young Lee,Franck Dernoncourt,http://arxiv.org/abs/1603.03827v1,http://arxiv.org/pdf/1603.03827v1
2,2,1606.00776v2,6,We introduce the multiresolution recurrent neu...,Multiresolution Recurrent Neural Networks: An ...,2016,cs.CL,cs.AI,cs.LG,Iulian Vlad Serban,Tim Klinger,http://arxiv.org/abs/1606.00776v2,http://arxiv.org/pdf/1606.00776v2
3,23,1705.08142v2,5,Multi-task learning is motivated by the observ...,Learning what to share between loosely related...,2017,stat.ML,cs.AI,cs.CL,Sebastian Ruder,Joachim Bingel,http://arxiv.org/abs/1705.08142v2,http://arxiv.org/pdf/1705.08142v2
4,7,1709.02349v2,9,We present MILABOT: a deep reinforcement learn...,A Deep Reinforcement Learning Chatbot,2017,cs.CL,cs.AI,cs.LG,Iulian V. Serban,Chinnadhurai Sankar,http://arxiv.org/abs/1709.02349v2,http://arxiv.org/pdf/1709.02349v2


In [177]:
#Get papers metadata from mongodb

#from pymongo import MongoClient
#client = MongoClient('mongodb://localhost:27017')
#db = client['Paper']
#collection = db['Papers']
#data = collection.find({})
#data_list = list(data)
#papers_df = pd.DataFrame(data_list)
#papers_df.drop(columns=['_id'],inplace=True)

In [178]:
#papers_df['title'] = papers_df['title'].str.lower()

In [179]:
papers_df['summary'][4]

'We present MILABOT: a deep reinforcement learning chatbot developed by the\nMontreal Institute for Learning Algorithms (MILA) for the Amazon Alexa Prize\ncompetition. MILABOT is capable of conversing with humans on popular small talk\ntopics through both speech and text. The system consists of an ensemble of\nnatural language generation and retrieval models, including template-based\nmodels, bag-of-words models, sequence-to-sequence neural network and latent\nvariable neural network models. By applying reinforcement learning to\ncrowdsourced data and real-world user interactions, the system has been trained\nto select an appropriate response from the models in its ensemble. The system\nhas been evaluated through A/B testing with real-world users, where it\nperformed significantly better than many competing systems. Due to its machine\nlearning architecture, the system is likely to improve with additional data.'

In [180]:
papers_df['summary'] = papers_df['summary'].str.replace('[{}]'.format(string.punctuation), '')

In [181]:
papers_df['summary'] = papers_df['summary'].str.replace('\d+|\n+','')

In [182]:
papers_df['summary'] = papers_df['summary'].str.strip()

In [183]:
#w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
#englishStemmer=SnowballStemmer("english") #define stemming dict

In [184]:
#def stemm_texts(text):
#   return [englishStemmer.stem(w) for w in w_tokenizer.tokenize(text)]

In [185]:
#papers_df['summary'] = papers_df.summary.apply(stemm_texts)

In [186]:
papers_df['summary'][4]

'We present MILABOT a deep reinforcement learning chatbot developed by theMontreal Institute for Learning Algorithms MILA for the Amazon Alexa Prizecompetition MILABOT is capable of conversing with humans on popular small talktopics through both speech and text The system consists of an ensemble ofnatural language generation and retrieval models including templatebasedmodels bagofwords models sequencetosequence neural network and latentvariable neural network models By applying reinforcement learning tocrowdsourced data and realworld user interactions the system has been trainedto select an appropriate response from the models in its ensemble The systemhas been evaluated through AB testing with realworld users where itperformed significantly better than many competing systems Due to its machinelearning architecture the system is likely to improve with additional data'

In [187]:
print("Papers : ", papers_df.shape)

Papers :  (8000, 13)


In [188]:
papers_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   day       8000 non-null   int64 
 1   id        8000 non-null   object
 2   month     8000 non-null   int64 
 3   summary   8000 non-null   object
 4   title     8000 non-null   object
 5   year      8000 non-null   int64 
 6   Topic1    8000 non-null   object
 7   Topic2    8000 non-null   object
 8   Topic3    4319 non-null   object
 9   Author1   8000 non-null   object
 10  Author2   8000 non-null   object
 11  textLink  8000 non-null   object
 12  pdfLink   8000 non-null   object
dtypes: int64(3), object(10)
memory usage: 812.6+ KB


In [189]:
papers_df.iloc[757].id

'cs/0204043v1'

In [190]:
# Reverse mapping of indices and paper titles
indices = pd.Series(papers_df.index, index=papers_df['title']).drop_duplicates()

In [191]:
indices

title
Dual Recurrent Attention Units for Visual Question Answering                                      0
Sequential Short-Text Classification with Recurrent and Convolutional  Neural Networks            1
Multiresolution Recurrent Neural Networks: An Application to Dialogue  Response Generation        2
Learning what to share between loosely related tasks                                              3
A Deep Reinforcement Learning Chatbot                                                             4
                                                                                               ... 
BreathRNNet: Breathing Based Authentication on Resource-Constrained IoT  Devices using RNNs    7995
Deep Learning applied to Road Traffic Speed forecasting                                        7996
ChainerMN: Scalable Distributed Deep Learning Framework                                        7997
Minimum Energy Quantized Neural Networks                                                      

In [192]:
start = time.time()

In [193]:
ind = indices['Semi-Supervised Learning with Ladder Networks']

In [194]:
tfv = TfidfVectorizer(max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{3,}',
            ngram_range=(1, 3),
            stop_words = 'english')


In [195]:
papers_df['all_content'] = papers_df['title']+papers_df['summary']
#anahtar kelime, tag, tarih(son 5 yıl )

In [196]:
tfv_matrix = tfv.fit_transform(papers_df['all_content'])

In [197]:
tfv_matrix.shape

(8000, 1266511)

In [198]:
#This function helps to find the most similar papers to specified paper.
def calc_similarity(method_name):
    
    if method_name == 'sigmoid_kernel':
        matrix = sigmoid_kernel(tfv_matrix, tfv_matrix,gamma = 0.8, coef0=0.5)
    elif method_name == 'linear_kernel':
        matrix = linear_kernel(tfv_matrix, tfv_matrix)
    elif method_name == 'euclidean_distances':
        matrix = euclidean_distances(tfv_matrix)
    elif method_name == 'cosine_similarity':
        matrix = cosine_similarity(tfv_matrix,tfv_matrix)
    elif method_name == 'pearsons_correlation':
        tfv_array = tfv_matrix.toarray()
        matrix = []
        for i in range(len(tfv_array)):
             matrix.append(pearsonr(tfv_array[ind], tfv_array[i])[0])
        
    
    return matrix

In [199]:
matrix = calc_similarity('cosine_similarity')

In [200]:
def give_rec(title, matrix=matrix):
    # Get the index corresponding to title
    idx = indices[title]

    # Get the pairwsie similarity scores 
    sig_scores = list(enumerate(matrix[idx]))

    # Sort the paper 
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)

    # Scores of the 10 most similar papers
    sig_scores = sig_scores[1:11]
    
    data = []
    count = 0
    while count<10:
        data.append([(papers_df.iloc[sig_scores[count][0]].id),(papers_df['title'].iloc[sig_scores[count][0]]), sig_scores[count][1]])
        count=count+1

    df = pd.DataFrame(data, columns = ["id", "title", "score"])
    return df

In [201]:
# Testing our content-based recommendation system with Genetic Algorithms and its use with back-propagation network

#str = 'Multiresolution Recurrent Neural Networks: An Application To Dialogue Response Generation'
give_rec('Semi-Supervised Learning with Ladder Networks').head(10)

,id,title,score
0,1504.08215v1,Lateral Connections in Denoising Autoencoders ...,0.112891
1,1611.02320v1,Adversarial Ladder Networks,0.054119
2,1711.07476v2,Virtual Adversarial Ladder Networks For Semi-s...,0.049742
3,1706.02124v2,Semi-Supervised Phoneme Recognition with Recur...,0.048963
4,1707.09219v4,Recurrent Ladder Networks,0.046477
5,1612.09030v2,Meta-Unsupervised-Learning: A supervised appro...,0.040736
6,1612.01942v1,Semi-Supervised Learning with the Deep Renderi...,0.038456
7,1711.04313v1,Semi-Supervised Learning via New Deep Network ...,0.036589
8,1210.5840v1,Supervised Learning with Similarity Functions,0.034717
9,1612.01756v3,Video Ladder Networks,0.031929


In [202]:
end = time.time()

In [203]:
print(end - start)

9.691247463226318
